## Instalando Bibliotecas
Lembre de retirar o '#' para "descomentar" essa linha, para realizar a instalação das bibliotecas <br>
necessárias, após instalado uma vez, pode comentar novamente.

In [ ]:
pip install openmeteo-requests requests-cache retry-requests openpyxl googletrans==4.0.0-rc1

## Imports
Realizando os imports necessários para usar as API's

In [2]:
import json
import requests
import pandas as pd
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from googletrans import Translator

## Questão 1:
Quantos feriados há no Brasil em todo o ano de 2024?

Usei a API Nager.DATE ( Public Holidays )<br>
Aqui usei a API e laço for para contabilizar a quantidade de feriados no ano de 2024

In [3]:
response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)
feriados_2024 = 0

for public_holiday in public_holidays:
  feriados_2024 += 1
  
print(f"Teremos {feriados_2024 - 1} feriados em 2024, porém teremos {feriados_2024} dias de feriado.")

Teremos 13 feriados em 2024, porém teremos 14 dias de feriado.


Resposta: Em 2024 teremos 13 feriados em 2024, porém teremos 14 dias de feriado, pois o carnaval está duplicado, pois são dois dias.

## Questão 2:
Qual mês de 2024 tem o maior número de feriados?

Nessa primeira parte eu decidi criar um DataFrame com os feriados, para que eu pudesse ir utilizando<br>
essa base de dados durante as questões.

In [4]:
df_feriados = pd.DataFrame(columns=['Holiday Name', 'Date'])
rows = []

for public_holiday in public_holidays:
  rows.append({'Holiday Name': public_holiday['name'], 'Nome Local': public_holiday['localName'],'Date': public_holiday['date'],})

translator = Translator()

def traduzir_texto(texto, src='en', dest='pt'):
    try:
        if texto is None:
            return 'Texto não disponível'
        traducao = translator.translate(texto, src=src, dest=dest)
        return traducao.text
    except Exception as e:
        print(f"Erro ao traduzir o texto: {e}")
        return 'Erro na tradução'



df_feriados = pd.concat([df_feriados, pd.DataFrame(rows)], ignore_index=True)
df_feriados['Date'] = pd.to_datetime(df_feriados['Date']) + pd.DateOffset(hours=3)
df_feriados['Month'] = df_feriados['Date'].dt.month_name()
df_feriados['Day'] = df_feriados['Date'].dt.weekday
df_feriados['Weekday'] = df_feriados['Date'].dt.day_name()

df_feriados['Weekday'] = df_feriados['Weekday'].apply(traduzir_texto)

Visualização do DataFrame

In [5]:
df_feriados.head(30)

,Holiday Name,Date,Nome Local,Month,Day,Weekday
0,New Year's Day,2024-01-01 03:00:00,Confraternização Universal,January,0,Segunda-feira
1,Carnival,2024-02-12 03:00:00,Carnaval,February,0,Segunda-feira
2,Carnival,2024-02-13 03:00:00,Carnaval,February,1,Terça-feira
3,Good Friday,2024-03-29 03:00:00,Sexta-feira Santa,March,4,Sexta-feira
4,Easter Sunday,2024-03-31 03:00:00,Domingo de Páscoa,March,6,Domingo
5,Tiradentes,2024-04-21 03:00:00,Dia de Tiradentes,April,6,Domingo
6,Labour Day,2024-05-01 03:00:00,Dia do Trabalhador,May,2,Quarta-feira
7,Corpus Christi,2024-05-30 03:00:00,Corpus Christi,May,3,Quinta-feira
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,Revolução Constitucionalista de 1932,July,1,Terça-feira
9,Independence Day,2024-09-07 03:00:00,Dia da Independência,September,5,Sábado


Aqui onde de fato eu fiz a captura dos meses que tinham mais feriados no ano de 2024.<br>
Usei dt.month para capturar apenas o mês da coluna Date, que foi passada para datetime anteriormente<br>
e como havia meses com a mesma quantidade de feriados, usei .max para pegar o valor máximo e printar<br>
todos que estavam no valor máximo

In [6]:
feriados_por_mes = df_feriados['Date'].dt.month.value_counts().sort_index()
max_feriados = feriados_por_mes.max()
meses_com_max_feriados = feriados_por_mes[feriados_por_mes == max_feriados]

print("Meses com o maior número de feriados:\n")
print(meses_com_max_feriados)

Meses com o maior número de feriados:

Date
2     2
3     2
5     2
11    2
Name: count, dtype: int64


Como fevereiro na realidade tem 1 feriado com 2 dias, então os meses com maior quantidade de feriados são : <br>
<br>
Resposta: <br>
3 - Março ----- 2 feriados <br>
5 - Maio ----- 2 feriados<br>
11 - Novembro ----- 2 feriados<br>

## Questão 3: 
Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?


Aqui eu aproveitei o DataFrame e fiz um laço for para percorrer a coluna dias, e verificar<br>
quantos dias ficam entre segunda e sexta (entre 0 e 4).

In [7]:
count_dias_semana = 0

for dia in df_feriados['Day']:
    if 0 <= dia <= 4:
        count_dias_semana += 1

In [8]:
print(count_dias_semana)

9


Resposta: 9 feriados

## Questão 4:
Qual foi a temperatura média em cada mês?

Através do site: [Open Meteo](https://open-meteo.com/en/docs/historical-weather-api#latitude=-22.9064&longitude=-43.1822&start_date=2024-01-01&end_date=2024-08-01&hourly=&daily=temperature_2m_mean&timezone=America%2FSao_Paulo) <br>
Eu consegui achar como realizar as configurar para trabalhar com a API. 

In [9]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822,
	"start_date": "2024-01-01",
	"end_date": "2024-08-01",
	"daily": ["weather_code", "temperature_2m_mean"],
	"timezone": "America/Sao_Paulo"
}

responses = openmeteo.weather_api(url, params=params)

response = responses[0]
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

daily_data = {"Date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = False),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = False),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_dataframe = pd.DataFrame(data = daily_data)

daily_data["weather_code"] = daily_weather_code
daily_data['Ano'] = daily_dataframe['Date'].dt.year
daily_data['Mês'] = daily_dataframe['Date'].dt.month
daily_data["Temperatura Média"] = daily_temperature_2m_mean



Tradução dos meses

In [10]:
meses = {
    1 : 'Janeiro',
    2 : 'Fevereiro',
    3 : 'Março',
    4 : 'Abril',
    5 : 'Maio',
    6 : 'Junho',
    7 : 'Julho',
    8 : 'Agosto',
    9 : 'Setembro',
    10: 'Outubro',
    11: 'Novembro',
    12: 'Dezembro'
}

In [11]:
daily_data['Nome do Mês'] = daily_dataframe['Date'].apply(lambda x: x.month).map(meses)
daily_dataframe = pd.DataFrame(data = daily_data)


daily_dataframe.head(50)

,Date,weather_code,Ano,Mês,Temperatura Média,Nome do Mês
0,2024-01-01 03:00:00,51.0,2024,1,24.854582,Janeiro
1,2024-01-02 03:00:00,51.0,2024,1,25.885834,Janeiro
2,2024-01-03 03:00:00,63.0,2024,1,26.223335,Janeiro
3,2024-01-04 03:00:00,61.0,2024,1,24.869164,Janeiro
4,2024-01-05 03:00:00,51.0,2024,1,25.183752,Janeiro
5,2024-01-06 03:00:00,51.0,2024,1,25.402502,Janeiro
6,2024-01-07 03:00:00,3.0,2024,1,25.983749,Janeiro
7,2024-01-08 03:00:00,1.0,2024,1,28.962919,Janeiro
8,2024-01-09 03:00:00,51.0,2024,1,29.317085,Janeiro
9,2024-01-10 03:00:00,53.0,2024,1,29.946245,Janeiro


Continuando a resolução, aqui eu criei um DataFrame para as médias mensais

Peguei as informações do outro DataFrame e agrupei os dados usando o mês como métrica, calculando a média de temperatura para cada mês

In [12]:
df_medias_mensais = pd.DataFrame(columns=['Mês', 'Ano', 'Temperatura Média'])

df_medias_mensais = daily_dataframe.groupby(['Ano','Mês' ,'Nome do Mês'])['Temperatura Média'].mean().reset_index()

df_medias_mensais.head(10)

,Ano,Mês,Nome do Mês,Temperatura Média
0,2024,1,Janeiro,26.694233
1,2024,2,Fevereiro,27.212988
2,2024,3,Março,26.416479
3,2024,4,Abril,25.169930
4,2024,5,Maio,24.990740
5,2024,6,Junho,22.648958
6,2024,7,Julho,21.105591
7,2024,8,Agosto,21.225414


Resposta: <br>

Janeiro: 26.69 ºC<br>
Fevereiro: 27.21 ºC<br>
Março: 26.41 ºC<br>
Abril: 25.16 ºC<br>
Maio: 24.99 ºC<br>
Junho: 22.64 ºC<br>
Julho: 21.10 ºC<br>
Agosto: 21,22 ºC<br>

## Questão 5:
Qual foi o tempo predominante em cada mês nesse período?

Aqui eu fiz uma função utilizando o método mode() para retornar o weather_code mais recorrente<br>
e agrupei usando Ano e Mês como filtro, aplicando a função em função do weather_code<br>
e fiz o merge do dataframe medias mensais e o novo dataframe criado

In [13]:
def weather_code_recorrente(valores):
    frequencia = valores.mode()
    return frequencia.iloc[0] if not frequencia.empty else None

weather_code_comum = daily_dataframe.groupby(['Ano', 'Mês'])['weather_code'].apply(weather_code_recorrente).reset_index()
weather_code_comum.rename(columns={'weather_code': 'Weather_Code_Mais_Recorrente'}, inplace=True)


df_resultado = pd.merge(df_medias_mensais, weather_code_comum, on=['Ano', 'Mês'])

df_resultado.head(10)

,Ano,Mês,Nome do Mês,Temperatura Média,Weather_Code_Mais_Recorrente
0,2024,1,Janeiro,26.694233,63.0
1,2024,2,Fevereiro,27.212988,51.0
2,2024,3,Março,26.416479,51.0
3,2024,4,Abril,25.169930,3.0
4,2024,5,Maio,24.990740,51.0
5,2024,6,Junho,22.648958,1.0
6,2024,7,Julho,21.105591,3.0
7,2024,8,Agosto,21.225414,2.0


Aqui eu "carreguei" o JSON ao nosso código e fiz uma função, para fazer o mapping<br>
do weather_code do meu DataFrame com os valores que estão no JSON.

In [14]:
with open('descriptions.json', 'r') as file:
    weather_code_dict = json.load(file)

def pegar_Descrição_JSON(code, time_of_day='day'):
    if pd.isna(code):  # Verifica se o código é NaN
        return 'Description Not Available'
    code_str = str(int(code))
    if code_str in weather_code_dict:
        descricao = weather_code_dict[code_str].get(time_of_day, {}).get('description', 'Descrição não disponível')
        descricao_str = str(descricao)
        print(f"Código: {code_str}, Descrição: {descricao_str}")  # Mensagem de depuração
        return descricao_str
    else:
        return 'Descrição não disponível'


df_resultado['Descrição do Tempo'] = df_resultado['Weather_Code_Mais_Recorrente'].apply(lambda x: pegar_Descrição_JSON(x, 'day'))
df_resultado['Descrição do Tempo'] = df_resultado['Descrição do Tempo'].astype(str)
df_resultado['Descrição do Tempo'] = df_resultado['Descrição do Tempo'].apply(traduzir_texto)

df_resultado.head(10)

Código: 63, Descrição: Rain
Código: 51, Descrição: Light Drizzle
Código: 51, Descrição: Light Drizzle
Código: 3, Descrição: Completely cloudy
Código: 51, Descrição: Light Drizzle
Código: 1, Descrição: Mainly Sunny
Código: 3, Descrição: Completely cloudy
Código: 2, Descrição: Partly Cloudy


,Ano,Mês,Nome do Mês,Temperatura Média,Weather_Code_Mais_Recorrente,Descrição do Tempo
0,2024,1,Janeiro,26.694233,63.0,Chuva
1,2024,2,Fevereiro,27.212988,51.0,Garoa leve
2,2024,3,Março,26.416479,51.0,Garoa leve
3,2024,4,Abril,25.169930,3.0,Completamente nublado
4,2024,5,Maio,24.990740,51.0,Garoa leve
5,2024,6,Junho,22.648958,1.0,Principalmente ensolarado
6,2024,7,Julho,21.105591,3.0,Completamente nublado
7,2024,8,Agosto,21.225414,2.0,Parcialmente nublado


Resposta: Foi tomado como base para todos os dias, a parte da manhã<br>
<br>
Janeiro - Chuva<br>
Fevereiro - Garoa Leve<br>
Março - Garoa Leve<br>
Abril - Completamente nublado<br>
Maio - Garoa Leve<br>
Junho - Principalmente ensolarado<br>
Julho - Completamente nublado<br>
Agosto - Parcialmente nublado<br>

## Questão 6:
Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

Aqui eu fiz o merge do dataframe das temperaturas com os feriados.

In [15]:
df_feriados_temp = pd.merge(df_feriados, daily_dataframe[['Date', 'Temperatura Média', 'weather_code']], on='Date', how='left')

Adicionei as Descrições do tempo

In [16]:
df_feriados_temp['Descrição do Tempo'] = df_feriados_temp['weather_code'].apply(lambda x: pegar_Descrição_JSON(x, 'day'))
df_feriados_temp['Descrição do Tempo'] = df_feriados_temp['Descrição do Tempo'].apply(traduzir_texto)

Código: 51, Descrição: Light Drizzle
Código: 1, Descrição: Mainly Sunny
Código: 51, Descrição: Light Drizzle
Código: 61, Descrição: Light Rain
Código: 53, Descrição: Normal Drizzle
Código: 2, Descrição: Partly Cloudy
Código: 0, Descrição: Completely Sunny
Código: 51, Descrição: Light Drizzle
Código: 53, Descrição: Normal Drizzle


In [17]:
df_feriados_temp.tail(15)

,Holiday Name,Date,Nome Local,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
0,New Year's Day,2024-01-01 03:00:00,Confraternização Universal,January,0,Segunda-feira,24.854582,51.0,Garoa leve
1,Carnival,2024-02-12 03:00:00,Carnaval,February,0,Segunda-feira,30.156668,1.0,Principalmente ensolarado
2,Carnival,2024-02-13 03:00:00,Carnaval,February,1,Terça-feira,30.531670,51.0,Garoa leve
3,Good Friday,2024-03-29 03:00:00,Sexta-feira Santa,March,4,Sexta-feira,25.044168,61.0,Chuva leve
4,Easter Sunday,2024-03-31 03:00:00,Domingo de Páscoa,March,6,Domingo,24.531664,53.0,Garoa normal
5,Tiradentes,2024-04-21 03:00:00,Dia de Tiradentes,April,6,Domingo,23.087919,2.0,Parcialmente nublado
6,Labour Day,2024-05-01 03:00:00,Dia do Trabalhador,May,2,Quarta-feira,28.108751,0.0,Completamente ensolarado
7,Corpus Christi,2024-05-30 03:00:00,Corpus Christi,May,3,Quinta-feira,20.779579,51.0,Garoa leve
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,Revolução Constitucionalista de 1932,July,1,Terça-feira,21.487915,53.0,Garoa normal
9,Independence Day,2024-09-07 03:00:00,Dia da Independência,September,5,Sábado,NaN,NaN,Descrição não disponível


Aqui eu fiz um filtro eu guardei em um novo dataframe

In [18]:
feriados_filtro = df_feriados_temp[
    (df_feriados_temp['Date'] >= '2024-01-01') &
    (df_feriados_temp['Date'] <= '2024-08-01')
]

feriados_filtro.head(14)

,Holiday Name,Date,Nome Local,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
0,New Year's Day,2024-01-01 03:00:00,Confraternização Universal,January,0,Segunda-feira,24.854582,51.0,Garoa leve
1,Carnival,2024-02-12 03:00:00,Carnaval,February,0,Segunda-feira,30.156668,1.0,Principalmente ensolarado
2,Carnival,2024-02-13 03:00:00,Carnaval,February,1,Terça-feira,30.531670,51.0,Garoa leve
3,Good Friday,2024-03-29 03:00:00,Sexta-feira Santa,March,4,Sexta-feira,25.044168,61.0,Chuva leve
4,Easter Sunday,2024-03-31 03:00:00,Domingo de Páscoa,March,6,Domingo,24.531664,53.0,Garoa normal
5,Tiradentes,2024-04-21 03:00:00,Dia de Tiradentes,April,6,Domingo,23.087919,2.0,Parcialmente nublado
6,Labour Day,2024-05-01 03:00:00,Dia do Trabalhador,May,2,Quarta-feira,28.108751,0.0,Completamente ensolarado
7,Corpus Christi,2024-05-30 03:00:00,Corpus Christi,May,3,Quinta-feira,20.779579,51.0,Garoa leve
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,Revolução Constitucionalista de 1932,July,1,Terça-feira,21.487915,53.0,Garoa normal


Resposta: <br>
Ano Novo : 24.84 ºC e Garoa Leve<br>
Carnaval (2024-02-12) : 30.15 ºC e Principalmente Ensolarado<br>
Carnaval (2024-02-13) : 30.53 ºC e Garoa Leve<br>
Sexta-feira Santa: 25.04 ºC e Chuva leve<br>
Páscoa: 24.53 ºC e Garoa normal<br>
Tiradentes: 23.08 ºC e Parcialmente nublado<br>
Dia do Trabalho: 28.10 ºC e Completamente ensolarado<br>
Corpus Christi: 20.77 ºC e Garoa leve<br>
Revolução Constitucionalista de 1932: 21.48 ºC e Garoa normal<br>

## Questão 7:
Considere as seguintes suposições:

- O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
- Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
- O cidadão carioca só vai à praia quando não está com frio;
- O cidadão carioca também só vai à praia em dias com sol, evitando dias totalmente nublados ou chuvosos (considere weather_code para determinar as condições climáticas).
<br>
<br>
Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?



Fiz um filtro utilizando com métrica temperatura abaixo de 20 e condição de tempo acima de 2 ( Parcialmente nublado )

In [19]:
feriados_ruins = df_feriados_temp[
    (df_feriados_temp['Temperatura Média'] < 20) | 
    (df_feriados_temp['weather_code'] > 2)
]
count_feriados_ruins = len(feriados_ruins)

print(f"Em 2024 teremos {count_feriados_ruins} feriados ruins para praia.")
feriados_ruins.head(15)

Em 2024 teremos 6 feriados ruins para praia.


,Holiday Name,Date,Nome Local,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
0,New Year's Day,2024-01-01 03:00:00,Confraternização Universal,January,0,Segunda-feira,24.854582,51.0,Garoa leve
2,Carnival,2024-02-13 03:00:00,Carnaval,February,1,Terça-feira,30.531670,51.0,Garoa leve
3,Good Friday,2024-03-29 03:00:00,Sexta-feira Santa,March,4,Sexta-feira,25.044168,61.0,Chuva leve
4,Easter Sunday,2024-03-31 03:00:00,Domingo de Páscoa,March,6,Domingo,24.531664,53.0,Garoa normal
7,Corpus Christi,2024-05-30 03:00:00,Corpus Christi,May,3,Quinta-feira,20.779579,51.0,Garoa leve
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,Revolução Constitucionalista de 1932,July,1,Terça-feira,21.487915,53.0,Garoa normal


Resposta:<br>

Eu usei como paramêtro as condições até no máximo parcialmente nublado, onde temos sol ainda<br>
assim, tiveram 6 feriados não aproveitáveis para o carioca

## Questão 8:
Qual foi o feriado "mais aproveitável" de 2024?

Aqui eu decidi ordenar por temperatura e weather_code, de forma que a ordenação por temperatura fosse decrescente e por weather_code<br>
fosse crescente. Após isso, eu filtrei o weather_code <= 2

In [30]:
feriado_mais_aproveital = df_feriados_temp.sort_values(by=['Temperatura Média', 'weather_code'], ascending=[False, True])

feriado_mais_aproveital.head(15)

feriado_aproveitavel_filtro = feriado_mais_aproveital[
    (feriado_mais_aproveital['weather_code'] <= 2)
]

linha_result = feriado_aproveitavel_filtro.head(1)
valor = linha_result['Nome Local'].values[0]

feriado_aproveitavel_filtro.head()

,Holiday Name,Date,Nome Local,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
1,Carnival,2024-02-12 03:00:00,Carnaval,February,0,Segunda-feira,30.156668,1.0,Principalmente ensolarado
6,Labour Day,2024-05-01 03:00:00,Dia do Trabalhador,May,2,Quarta-feira,28.108751,0.0,Completamente ensolarado
5,Tiradentes,2024-04-21 03:00:00,Dia de Tiradentes,April,6,Domingo,23.087919,2.0,Parcialmente nublado


Dessa forma, o feriado mais aproveitavel foi o dia do trabalhador, já que houve um dia completamente ensolarado, e a diferença de temperatura para o carnaval foi pouca, menos de 2ºC. Lembrando também, que no dia seguinte do carnaval teve garoa leve, ou seja, havia o risco de tempo ruim no Carnaval do dia 12-02-2024. Dessa forma, no dia do trabalhador é um dia perfeito para aproveitar a praia e a natureza.

## Integração com BigQuery para alimentar o PowerBI

Instalando biblioteca para me conectar ao bigquery

In [21]:
#pip install google-cloud-bigquery pandas-gbq google-cloud-iam


Imports necessários

In [22]:
# from google.cloud import bigquery
# from google.oauth2 import service_account
# from pandas_gbq import to_gbq

Criei uma função para pegar o link da imagem que corresponde ao clima do dia.

In [23]:
# def pegar_imagem_JSON(code, time_of_day='day'):
#     if pd.isna(code):  # Verifica se o código é NaN
#         return 'Imagem não disponível'
#     code_str = str(int(code))
#     if code_str in weather_code_dict:
#         image_link = weather_code_dict[code_str].get(time_of_day, {}).get('image', 'Imagem não disponível')
#         image_link = str(image_link)
#         print(f"Código: {code_str}, Link da Imagem: {image_link}")  # Mensagem de depuração
#         return image_link
#     else:
#         return 'Descrição não disponível'

Adicionei no dataframe feriados_temp

In [24]:
# df_feriados_temp['Link da Imagem do tempo'] = df_feriados_temp['weather_code'].apply(lambda x: pegar_imagem_JSON(x, 'day'))
# df_feriados_temp['id'] = df_feriados_temp['Nome Local'] + ' - ' + df_feriados_temp['Date'].dt.day.astype(str) + '/' + df_feriados_temp['Date'].dt.month.astype(str) + '/' + df_feriados_temp['Date'].dt.year.astype(str)


# df_feriados_temp.head(15)

Em credentials, você deve colocar onde está sua chave de autenticação da conta de serviço.

Criei uma tabala no BigQuery, usando o dataframe de feriados com temperatura.

In [25]:

# credentials = service_account.Credentials.from_service_account_file('C:/Users/cs134/OneDrive/Área de Trabalho/processo_seletivo/CHAVE/sinuous-gist-433018-f3-12d7ffa7d55f.json')

# # Substitua os valores conforme necessário
# project_id = 'sinuous-gist-433018-f3'
# table_id_feriados = 'desafiovaga.feriados'

# # Carregue o DataFrame para o BigQuery
# to_gbq(df_feriados_temp, table_id_feriados, project_id=project_id, if_exists="replace", credentials=credentials)


Criando novo dataframe das temperaturas, com as informações completas.

In [26]:
# temperaturas = daily_dataframe
# temperaturas_mes = df_resultado

# temperaturas['Descrição de Tempo'] = temperaturas['weather_code'].apply(lambda x: pegar_Descrição_JSON(x, 'day'))
# temperaturas['Link da Imagem do tempo'] = temperaturas['weather_code'].apply(lambda x: pegar_imagem_JSON(x, 'day'))
# temperaturas_mes['Link da Imagem do tempo'] = temperaturas_mes['Weather_Code_Mais_Recorrente'].apply(lambda x: pegar_imagem_JSON(x, 'day'))
# temperaturas['Descrição de Tempo'] = temperaturas['Descrição de Tempo'].apply(lambda x: traduzir_texto(x))

# temperaturas_mes.head(50)

In [27]:
#temperaturas.head(190)

Subindo tabela temperaturas para o BigQuery

In [28]:
# table_id_temperaturas = 'desafiovaga.temperaturas'

# to_gbq(temperaturas, table_id_temperaturas, project_id=project_id, if_exists="replace", credentials=credentials)

Subindo Temperaturas_mes para o BigQuery

In [29]:
# table_id_temperaturas_mes = 'desafiovaga.temperaturas_mes'

# to_gbq(temperaturas_mes, table_id_temperaturas_mes, project_id=project_id, if_exists="replace", credentials=credentials)